In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import time
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

block_size = 8
batch_size = 4
max_ters = 10000
learning_rate=3e-4
evar_iters =250

cpu


In [8]:
with open ("/Users/sebascordova/Desktop/Code proyects/wizard_of_oz.txt", 'r', encoding = 'utf-8') as f:
    text = f.read()

chars = sorted(set(text))
print(chars)
print(len(chars))
vocab_size = len(chars)

['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '•', '™', '\ufeff']
92


In [14]:
tring_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
# print(data[:100])

In [4]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

block_size = 8

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    contexto = x[:t+1]
    tarjet = y[t]
    print('When input is', contexto, ' tarjet is ', tarjet)

When input is tensor([91])  tarjet is  tensor(48)
When input is tensor([91, 48])  tarjet is  tensor(65)
When input is tensor([91, 48, 65])  tarjet is  tensor(62)
When input is tensor([91, 48, 65, 62])  tarjet is  tensor(1)
When input is tensor([91, 48, 65, 62,  1])  tarjet is  tensor(44)
When input is tensor([91, 48, 65, 62,  1, 44])  tarjet is  tensor(75)
When input is tensor([91, 48, 65, 62,  1, 44, 75])  tarjet is  tensor(72)
When input is tensor([91, 48, 65, 62,  1, 44, 75, 72])  tarjet is  tensor(67)


In [10]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]


def get_batch(split):

    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    print(ix)

    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+batch_size +1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x,y

x,y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)


tensor([121606,  16811, 182658,  73795])
inputs:
tensor([[ 0, 51, 66, 77, 65, 72, 78, 77],
        [64, 66, 60, 14,  1, 44, 62, 75],
        [ 0,  3, 35, 72, 72, 61, 71, 62],
        [70,  1, 77, 65, 62,  0, 49, 71]])
targets:
tensor([[51, 66, 77, 65],
        [66, 60, 14,  1],
        [ 3, 35, 72, 72],
        [ 1, 77, 65, 62]])


In [15]:
class BiogramLanguagemodel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)


    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss


    def generate(self, index, max_new_tokens):

        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:,-1,:]
            probs = F.softmax(logits, dim = -1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index

model = BiogramLanguagemodel(vocab_size)
m = model.to(device)


context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



8“’T.c: :a)u™H5,uR3p&y#yUOsS%/1(dS#—”L4zH yG(_PHn26Po•(7r9bBHcAeP‘G/Z6N’"’-PwGBSY*E3oz'P4ke659/./QV™i—I]f“8$l]]#Mx&7"Y:A]v:S”“qn0T&RW*8if]nKTNdL'k: p™tVo“M,aR3E/1’4m&F&ymupMsqlPmphoe y•(AMY 'u$63xouBk%•l8n*Et k4GUZHfHmO—RD6X﻿n?m•)xTT3[6HWxqqKjV[-HY“7ONzdo7cL—dsB—Jy)R..)pUq4iZ]p
“H•o_I#.([6Mu•c?oqxdZwJGo5Z;*QQTlTk)Rt#qHIZ:jnQj8s8wBF)OqhQB3?(PmcJO$﻿3k:lwG#RiKBuS;™U$)!j4t[h(,DZ!!PEeqO”“)sAHtq #Q.#’”™UbNh#s4Hd—IZwu?uABF
P);﻿“ V-Eg:#q$F"rR]FFNy”QqC6T/_:kz— #'gMy[6v(S%O’6.c-Y™OD’OVP[)N2!.G]ZIM%e2f2A14
